## Set Up
- Create a (virtual) development environment to make Python version and packages manageable.  
- Import public datasets from Bigquery.

### Miniconda
Download Miniconda installer from the official site [Anaconda](https://www.anaconda.com/download).   
Use Miniconda command prompt to create/activate a new environment specifying Python version and packages to install. 

**Name the environment and choose a Python version.**  
`conda create -n my_env python=3.11`      
**List all created environments.**   
`conda env list`  
**Activate an environment by name.**  
`conda activate my_env`  
**Install essential packages for data analysis.**
```bash
conda install -c conda-forge \
  notebook \
  pandas \
  numpy \
  matplotlib \
  seaborn \
  scipy \
  scikit-learn
```
- Jupyter Notebook: `notebook`
- Data process & numeric operation: `pandas`, `numpy`
- Visulization: `matplotlib`, `seaborn`
- Statistics: `scipy`
- Machine learning: `scikit-learn`

**List all packages installed(dependencies not included) in the current environment**  
`conda env export --from-history`

**Add the environment as a Jupyter kernel**  
Install the IPython kernel package and register your environment as a Jupyter Notebook kernel.
```
conda install ipykernel
python -m ipykernel install --user --name my_env --display-name "Python (my_env)"

```
**Launch Jupyter notebook**  
`jupyter notebook`  
Select the desired kernel in Jupyter notebook interface.

In [4]:
conda env list


# conda environments:
#
base                   /opt/miniconda3
my_env               * /opt/miniconda3/envs/my_env


Note: you may need to restart the kernel to use updated packages.


### Connect to Bigquery
Install gcloud CLI with [offical guide](https://cloud.google.com/sdk/docs/install)(system-wide) and set up a project(to authorize the use of public datasets from Bigquery).  
Install and use `bigframe` package to import [New York Citi Bike dataset](https://console.cloud.google.com/marketplace/product/city-of-new-york/nyc-citi-bike?hl=en&inv=1&invt=AbxWgg) from Bigquery.  

**Create a new project in gcloud.**  
`gcloud projects create data-ana-0`  
**List all available projects.**  
`gcloud projects list`  
**Initialize a project.**  
`gcloud init`  
Confirm conversations from CLI.

**Install bigframe via conda.**  
`conda install conda-forge::bigframes`  
Need to restart the kernel.

In [4]:
import bigframes.pandas as bpd

PROJECT_ID = "data-ana-0"
bpd.options.bigquery.project = PROJECT_ID

sql_query = """
SELECT * 
FROM `bigquery-public-data.new_york.citibike_trips`
LIMIT 1000
"""

df = bpd.read_gbq(sql_query)
df.head()

,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,405,2016-08-04 08:07:33+00:00,2016-08-04 08:14:18+00:00,520,W 52 St & 5 Ave,40.759923,-73.976485,305,E 58 St & 3 Ave,40.760958,-73.967245,17789,Subscriber,1973,male
1,1018,2013-07-11 14:14:05+00:00,2013-07-11 14:31:03+00:00,520,W 52 St & 5 Ave,40.759923,-73.976485,442,W 27 St & 7 Ave,40.746647,-73.993915,19159,Subscriber,1967,female
2,2080,2016-09-09 17:35:27+00:00,2016-09-09 18:10:08+00:00,520,W 52 St & 5 Ave,40.759923,-73.976485,415,Pearl St & Hanover Square,40.704718,-74.00926,22270,Subscriber,1988,male
3,659,2013-10-28 16:44:28+00:00,2013-10-28 16:55:27+00:00,520,W 52 St & 5 Ave,40.759923,-73.976485,285,Broadway & E 14 St,40.734546,-73.990741,15775,Subscriber,1995,female
4,718,2016-08-07 17:46:03+00:00,2016-08-07 17:58:01+00:00,520,W 52 St & 5 Ave,40.759923,-73.976485,492,W 33 St & 7 Ave,40.7502,-73.990931,20344,Subscriber,1966,female


In [11]:
sql_query = """
SELECT DISTINCT end_station_name
FROM `bigquery-public-data.new_york_citibike.citibike_trips`
WHERE end_station_id = 520
ORDER BY end_station_name
"""
df = bpd.read_gbq(sql_query)
df

,end_station_name
0,W 52 St & 5 Ave


By comparing the "start_station_id"/"start_station_name" and "end_station_id"/"end_station_name", we can draw conclusion that the id for the starting stations and the ending stations are from the same indeces.

## Exploratory Data Analysis
By examining the "Details" tab of the `citibike_trips` table from the "new_york_citibike" dataset in Bigquery studio, I found that the table consists of:  
- 58,937,715 rows in total and it takes 7.47 GB to store.

While examining the "Schema" tab along with first few entries of the table returned above gives information that the table has:  
- 16 columns.

each row represents one trip using the citibike including:  
- start/end time,  
- start/end location,  
- bike id,  
- user information...

### Span of time of the entries

In [7]:
sql_query = """
SELECT
  MIN(starttime) AS earliest_trip,
  MAX(starttime) AS latest_trip
FROM
  `bigquery-public-data.new_york_citibike.citibike_trips`
"""
df = bpd.read_gbq(sql_query)
df

,earliest_trip,latest_trip
0,2013-07-01 00:00:00,2018-05-31 23:59:59.606000


By taking the min/max of the starttime column, we know that the entries from the target table starts from July 2013 to May 2018, roughly **5 years**.  
If we take the average, there are **11,787,543(10 million) trips recorded every year** and **982,295(1 million) trips recorded each month**.

### List of stations
*) Since I found the result of SQL query returned by `bigframe` troubled to display the dataframe in order, I use Bigquery UI to run SQL queries in the next sessions and save the results locally for further exploration.

**SQL query to get a full list of all unique Citibike stations**  
```sql
SELECT
  station_id,
  station_name,
  station_latitude,
  station_longitude
FROM (
  SELECT
    start_station_id AS station_id,
    start_station_name AS station_name,
    start_station_latitude AS station_latitude,
    start_station_longitude AS station_longitude
  FROM `bigquery-public-data.new_york_citibike.citibike_trips`
  WHERE start_station_id IS NOT NULL

  UNION DISTINCT

  SELECT
    end_station_id AS station_id,
    end_station_name AS station_name,
    end_station_latitude AS station_latitude,
    end_station_longitude AS station_longitude
  FROM `bigquery-public-data.new_york_citibike.citibike_trips`
  WHERE end_station_id IS NOT NULL
)
ORDER BY station_id

```

In [1]:
import pandas as pd

df_stations = pd.read_csv('./views/stations.csv')
df_stations.head(10)

,station_id,station_name,station_latitude,station_longitude
0,72,W 52 St & 11 Ave,40.767272,-73.993929
1,79,Franklin St & W Broadway,40.719116,-74.006667
2,82,St James Pl & Pearl St,40.711174,-74.000165
3,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323
4,116,W 17 St & 8 Ave,40.741776,-74.001497
5,119,Park Ave & St Edwards St,40.696089,-73.978034
6,120,Lexington Ave & Classon Ave,40.686768,-73.959282
7,127,Barrow St & Hudson St,40.731724,-74.006744
8,128,MacDougal St & Prince St,40.727103,-74.002971
9,137,E 56 St & Madison Ave,40.761628,-73.972924


In [18]:
df_stations.shape

(1018, 4)

There are 1018 Citibike stations, I didn't manually check all the rows for duplicates.  
So technically there will be 1018 * (1018 -1) = **1 million combinations of station pairs(directed)**.

## Visulization of the Flow between Stations
**Install Folium**  
`conda install folium -c conda-forge`  


### Flow on the day  of 2018-05-15(Tue)
**Query trip data for the day**  
```sql
SELECT
  start_station_id,
  start_station_name,
  start_station_latitude,
  start_station_longitude,
  end_station_id,
  end_station_name,
  end_station_latitude,
  end_station_longitude,
  COUNT(*) AS num_trips
FROM
  `bigquery-public-data.new_york_citibike.citibike_trips`
WHERE
  DATE(starttime) = '2018-05-15'
  AND start_station_id != end_station_id  -- Exclude round trips
GROUP BY 1,2,3,4,5,6,7,8
ORDER BY num_trips DESC

```

In [2]:
df_trips = pd.read_csv('./views/trips-2018-05-15.csv')
df_trips.head(10)

,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,num_trips
0,2006,Central Park S & 6 Ave,40.765909,-73.976342,3282,5 Ave & E 88 St,40.783070,-73.959390,23
1,519,Pershing Square North,40.751873,-73.977706,491,E 24 St & Park Ave S,40.740964,-73.986022,22
2,460,S 4 St & Wythe Ave,40.712859,-73.965903,3093,N 6 St & Bedford Ave,40.717452,-73.958509,21
3,514,12 Ave & W 40 St,40.760875,-74.002777,426,West St & Chambers St,40.717548,-74.013221,21
4,465,Broadway & W 41 St,40.755136,-73.986580,519,Pershing Square North,40.751873,-73.977706,19
5,435,W 21 St & 6 Ave,40.741740,-73.994156,462,W 22 St & 10 Ave,40.746920,-74.004519,18
6,2006,Central Park S & 6 Ave,40.765909,-73.976342,3165,Central Park West & W 72 St,40.775794,-73.976206,18
7,492,W 33 St & 7 Ave,40.750200,-73.990931,519,Pershing Square North,40.751873,-73.977706,17
8,519,Pershing Square North,40.751873,-73.977706,492,W 33 St & 7 Ave,40.750200,-73.990931,17
9,462,W 22 St & 10 Ave,40.746920,-74.004519,453,W 22 St & 8 Ave,40.744751,-73.999154,17


### Aggregate the trips df to num of trips on station-pairs(A to B is same as B to A)

In [8]:
# Create a normalized pair key
df_trips['pair'] = df_trips.apply(lambda row: tuple(sorted([row['start_station_id'], row['end_station_id']])), axis=1)
# Aggregate by pair
agg_df = df_trips.groupby('pair').agg({'num_trips': 'sum'}).reset_index()
# Extract station IDs
agg_df['station_id_1'] = agg_df['pair'].apply(lambda x: x[0])
agg_df['station_id_2'] = agg_df['pair'].apply(lambda x: x[1])
# Build a mapping from station_id to its info
station_info = {}
for _, row in df_trips.iterrows():
    station_info[row['start_station_id']] = {
        'name': row['start_station_name'],
        'latitude': row['start_station_latitude'],
        'longitude': row['start_station_longitude']
    }
    station_info[row['end_station_id']] = {
        'name': row['end_station_name'],
        'latitude': row['end_station_latitude'],
        'longitude': row['end_station_longitude']
    }

# Attach info for both stations in each pair
agg_df['station_1_name'] = agg_df['station_id_1'].map(lambda x: station_info[x]['name'])
agg_df['station_1_latitude'] = agg_df['station_id_1'].map(lambda x: station_info[x]['latitude'])
agg_df['station_1_longitude'] = agg_df['station_id_1'].map(lambda x: station_info[x]['longitude'])
agg_df['station_2_name'] = agg_df['station_id_2'].map(lambda x: station_info[x]['name'])
agg_df['station_2_latitude'] = agg_df['station_id_2'].map(lambda x: station_info[x]['latitude'])
agg_df['station_2_longitude'] = agg_df['station_id_2'].map(lambda x: station_info[x]['longitude'])

agg_df.to_csv('views/trips-2018-05-15-A.csv')
agg_df.head()

,pair,num_trips,station_id_1,station_id_2,station_1_name,station_1_latitude,station_1_longitude,station_2_name,station_2_latitude,station_2_longitude
0,"(72, 79)",2,72,79,W 52 St & 11 Ave,40.767272,-73.993929,Franklin St & W Broadway,40.719116,-74.006667
1,"(72, 127)",3,72,127,W 52 St & 11 Ave,40.767272,-73.993929,Barrow St & Hudson St,40.731724,-74.006744
2,"(72, 161)",1,72,161,W 52 St & 11 Ave,40.767272,-73.993929,LaGuardia Pl & W 3 St,40.729170,-73.998102
3,"(72, 173)",4,72,173,W 52 St & 11 Ave,40.767272,-73.993929,Broadway & W 49 St,40.760683,-73.984527
4,"(72, 195)",1,72,195,W 52 St & 11 Ave,40.767272,-73.993929,Liberty St & Broadway,40.709056,-74.010434


In [20]:
df_trips = df_trips[:500] # Only top 500 rows(trips) are included for the visulization because loading the complete table broke my notebook session.

In [21]:
import folium
from folium.plugins import HeatMap, AntPath

# Base map centered on NYC
m = folium.Map(location=[40.7128, -74.0060], zoom_start=13, tiles='cartodbpositron')

# Add station markers based on 'stations.csv'
for _, row in df_stations.iterrows():
    folium.CircleMarker(
        location=[row['station_latitude'], row['station_longitude']],
        radius=3,
        color='blue',
        fill=True,
        tooltip=f"{row['station_name']} (ID: {row['station_id']})"
    ).add_to(m)

# Add flow lines between stations based on 'trip-2018-05-15.csv'
for _, row in df.iterrows():
    AntPath(
        locations=[
            [row['start_station_latitude'], row['start_station_longitude']],
            [row['end_station_latitude'], row['end_station_longitude']]
        ],
        color='red' if row['num_trips'] > 16 else 'orange',
        weight=row['num_trips']/3,  # Scale line thickness
        dash_array='5,5',
        tooltip=f"{row['num_trips']} trips"
    ).add_to(m)

m.save('./outputs/flows-2018-05-15-C.html')

In [22]:
from IPython.display import IFrame

IFrame('./outputs/flows-2018-05-15-C.html', width=800, height=600)

The map of the New York city is rendered. Blue circles(not trypophobic-friendly) are stations between which flows are marked as straight lines(red for busy, orange for normal).  
The thickness of the flow lines are corresponded to the number of trips(biker flow) and they are animated indicating the direction of trips between the two stations.

**Notes-2025-05-16**
- Take the log of the num of trips for the thickness with normalization.  
- Don't use ant path for large numbers of entries.  
- Make the size of station marker correpond to the flows that it processes.  


### Aggregate the trips dataframe to count the number of trips start/end with each station

In [21]:
# aggregate trips start with each station
start_agg = df_trips.groupby(
    ['start_station_id', 'start_station_name', 'start_station_latitude', 'start_station_longitude']
)['num_trips'].sum().reset_index()
start_agg = start_agg.rename(columns={
    'start_station_id': 'station_id',
    'start_station_name': 'station_name',
    'start_station_latitude': 'station_latitude',
    'start_station_longitude': 'station_longitude',
    'num_trips': 'start_trips'
})
# aggregate trips end with each station
end_agg = df_trips.groupby(
    ['end_station_id', 'end_station_name', 'end_station_latitude', 'end_station_longitude']
)['num_trips'].sum().reset_index()

end_agg = end_agg.rename(columns={
    'end_station_id': 'station_id',
    'end_station_name': 'station_name',
    'end_station_latitude': 'station_latitude',
    'end_station_longitude': 'station_longitude',
    'num_trips': 'end_trips'
})
# merge the start/end aggregated dfs
agg_df = pd.merge(
    start_agg,
    end_agg,
    on=['station_id', 'station_name', 'station_latitude', 'station_longitude'],
    how='outer'
).fillna(0)
agg_df['num_trips'] = agg_df['start_trips'] + agg_df['end_trips']
agg_df = agg_df[['station_id', 'station_name', 'station_latitude', 'station_longitude', 'num_trips']]
agg_df['num_trips'] = agg_df['num_trips'].astype(int)
# add stations with 0 trips from df_stations
agg_df = pd.merge(
    df_stations,
    agg_df[['station_id', 'num_trips']],
    on='station_id',
    how='left'
)
# For stations not in agg_df, set num_trips to 0
agg_df['num_trips'] = agg_df['num_trips'].fillna(0).astype(int)

agg_df.to_csv('./views/trips-by-station-B.csv')
agg_df.head()

,station_id,station_name,station_latitude,station_longitude,num_trips
0,72,W 52 St & 11 Ave,40.767272,-73.993929,227
1,79,Franklin St & W Broadway,40.719116,-74.006667,155
2,82,St James Pl & Pearl St,40.711174,-74.000165,60
3,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,102
4,116,W 17 St & 8 Ave,40.741776,-74.001497,0


In [9]:
import pandas as pd
import folium

# load dataframes
df_stations = pd.read_csv('./views/stations.csv')
df_trips = pd.read_csv('./views/trips-2018-05-15-A.csv')
df_trips_by_station = pd.read_csv('./views/trips-by-station-B.csv')
# df_trips = df_trips[:500]

# Create base map centered on NYC
m = folium.Map(location=[40.7128, -74.0060], zoom_start=13, tiles='cartodbpositron')

# Add station markers
# Get min/max trip counts for scaling
min_trips = df_trips_by_station['num_trips'].min()
max_trips = df_trips_by_station['num_trips'].max()
for _, row in df_trips_by_station.iterrows():
    # Normalize trip count between 0-1
    normalized = (row['num_trips'] - min_trips) / (max_trips - min_trips if max_trips != min_trips else 1)
    folium.CircleMarker(
        location=[row['station_latitude'], row['station_longitude']],
        radius=2 + 7 * normalized,  # 2px (min) to 8px (max)
        color='#3186cc',
        opacity=0.2 + 0.5 * normalized,
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.2 + 0.5 * normalized,  # 20% to 70% opacity
        tooltip=(
            f"{row['station_name']}<br>"
            f"ID: {row['station_id']}<br>"
            f"Total trips: {row['num_trips']}"
        )
    ).add_to(m)

# Add flow lines with PolyLine
min_trips = df_trips['num_trips'].min()
max_trips = df_trips['num_trips'].max()
for _, row in df_trips.iterrows():
    normalized = (row['num_trips'] - min_trips) / (max_trips - min_trips if max_trips != min_trips else 1)
    folium.PolyLine(
        locations=[
            [row['station_1_latitude'], row['station_1_longitude']],
            [row['station_2_latitude'], row['station_2_longitude']]
        ],
        color='#ff7800' if row['num_trips'] > 16 else '#ffb380',  # Orange gradient
        weight=row['num_trips']/10,  # Scale the thickness of line
        opacity=normalized,
        tooltip=f"{row['num_trips']} trips"
    ).add_to(m)

m.save('./outputs/flows-2018-05-15-E.html')

In [10]:
from IPython.display import IFrame

IFrame('./outputs/flows-2018-05-15-E.html', width=800, height=600)

**Next steps**
- Use Google map api to plot the actual possible itineray of flows.
- Add a timeline slider to select and see the traffic of each day on one map.
- Add hourly stats of each flow/stations on the pop-up window when hovering cursor upon.